In [67]:
# 경쟁사 비교 에이전트

# 역할 : 경쟁 구도, 차별성 분석
# 내용 : 경쟁사 정보 검색 및 비교 분석

# 입력 : 기업명

In [68]:
# import

from langchain_teddynote.tools.tavily import TavilySearch

from typing import Annotated, TypedDict
from langgraph.graph.message import add_messages

from langchain_openai import ChatOpenAI
from langchain_teddynote.evaluator import GroundednessChecker
from langchain_teddynote.messages import messages_to_history

from langgraph.graph import END, StateGraph
from langgraph.checkpoint.memory import MemorySaver

from langchain_core.runnables import RunnableConfig
from langchain_teddynote.messages import stream_graph, random_uuid
from langgraph.graph import StateGraph, START, END

from langchain.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.messages import BaseMessage
import os

In [69]:
#!pip install langchain-teddynote

In [71]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [72]:
# 그래프 상태
class WebSearchState(TypedDict):  
    question: Annotated[str, "Question"] # 질문
    context: Annotated[str, "Context"]    
    answer: Annotated[str, "Answer"]        # 답변  
    messages: Annotated[list[BaseMessage], "Messages"] # 메시지(누적되는 list)  
    relevance: Annotated[str, "Relevance"]  # 관련성  

In [73]:
class AgentState(TypedDict):
    domain: Annotated[str, "Domain"]
    country: Annotated[str, "Country"]

    startup_list: Annotated[list[str], "Startup_list"]   # 스타트업 탐색 에이전트가 생성하는 주요 기업명 목록
    startup_profiles: Annotated[dict[str, dict], "Startup_profiles"]   # 스타트업별 정보 종합 저장소
    tech_summary: Annotated[dict[str, str], "Tech_summary"]  # 각 스타트업 기술 요약 정보
    founder_reputation: Annotated[dict[str, dict], "Founder_reputation"]  # 창업자 이력 + 평판 정보
    market_analysis: Annotated[dict[str, dict], "Market_analysis"]
    legal_risk: Annotated[dict[str, str], "Legal_risk"]
    competitor_info: Annotated[dict[str, dict], "Competitor_info"] # 경쟁자 정보
    investment_decision: Annotated[dict[str, str], "Investment_decision"]
    final_report: Annotated[str, "Final_report"]

In [ ]:
def relevance_check(state: WebSearchState) -> WebSearchState:
    # LLM을 직접 사용하여 관련성 체크
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    # 관련성 체크 프롬프트
    prompt = f"""
    당신은 질문과 검색 결과의 관련성을 평가하는 평가자입니다.
    
    질문: {state["question"]}
    
    검색 결과: {state["context"]}
    
    위 질문과 검색 결과가 서로 관련이 있는지 판단해주세요. 
    관련성이 있다면 "yes"를, 없다면 "no"를 응답해주세요.
    오직 "yes" 또는 "no"로만 대답하세요.
    """
    
    # LLM 실행
    response = llm.invoke(prompt)
    result = response.content.strip().lower()
    
    # "yes"면 True, "no"면 False 반환
    is_relevant = result == "yes"
    return WebSearchState(relevance=is_relevant)

def is_relevant(state: WebSearchState) -> str:
    """조건부 라우팅을 위한 함수"""
    if state["relevance"]:
        return "yes"
    else:
        return "no"

In [75]:
def llm_answer(state: WebSearchState) -> WebSearchState:
    latest_question = state["question"]
    context = state["context"]  # 웹 검색 결과로 받은 경쟁사 분석 정보

    # 경쟁사 분석 결과를 요약하고 핵심 인사이트를 추출하는 프롬프트
    report_prompt = f"""
        다음은 {latest_question}에 대한 경쟁사 분석 정보입니다. 이 내용을 기반으로 요약 보고서를 작성해 주세요.

        형식:
        1. 국내외 경쟁사 리스트 정리
        2. 핵심 인사이트 정리
        3. 경쟁사 대비 전략적 시사점
        4. 투자측면에서 이점

        경쟁사 분석 정보: {context}
        """

    response = llm.invoke(report_prompt)
    
    # 생성된 답변과 (유저의 질문, 답변) 메시지를 상태에 저장
    return WebSearchState(
        answer=response, 
        messages=[("user", latest_question), ("assistant", response)]
    )


In [ ]:
def web_search(state: WebSearchState) -> WebSearchState:
    
    tavily_tool = TavilySearch()
    company_name = state["question"]  # 'question' 키에서 기업명 추출
    search_query = f"{company_name} 경쟁사 비교 분석"
    
    # 최근 한 달의 결과 5개 가져오기
    search_result = tavily_tool.search(
        query=search_query,
        topic="general",    # 'general'로 변경 (유효한 옵션)
        days=30,            # 최근 한 달 데이터로 확장
        max_results=3,      # 결과 수 증가
        format_output=True, # 결과 포맷팅
    )

    if search_result:
        # 결과 헤더 추가하여 컨텍스트 명확히 하기
        formatted_result = f"## {company_name} 경쟁사 분석 정보\n\n" + "\n\n".join(search_result)
        return WebSearchState(context=formatted_result)
    else:
        print("No search results found.")
        return WebSearchState(context=f"{company_name}에 대한 경쟁사 비교 정보를 찾을 수 없습니다.")


In [77]:
def makeGraph():
    # 워크 플로우   
    workflow = StateGraph(WebSearchState)

    workflow.add_node("web_search", web_search)
    workflow.add_node("relevance_check", relevance_check)
    workflow.add_node("llm_answer", llm_answer)

    workflow.add_edge(START, "web_search")
    workflow.add_edge("web_search", "relevance_check")  
    workflow.add_edge("llm_answer", END)  

    workflow.add_conditional_edges(
        "relevance_check",  
        is_relevant,
        {
            "yes": "llm_answer", 
            "no": "web_search",  
        },
    )

    workflow.set_entry_point("web_search")

    memory = MemorySaver()
    return workflow.compile(checkpointer=memory)

In [87]:
def competitor_analysis(company : str):

    app = makeGraph()
    config = RunnableConfig(recursion_limit=10, configurable={"thread_id": random_uuid()})
    inputs = WebSearchState(
        question=company,
        context="",  # context 비워두기
        answer="",   # 초기값 비워두기
        messages=[], # 빈 메시지 목록
        relevance="",  # 초기값 비워두기
        usage_metadata={}  # usage_metadata를 빈 딕셔너리로 추가
    )

    result = app.invoke(inputs, config)
    return result['answer'].content

company = input("기업 이름 입력 : ") # 이부분 바꿔서 병렬로 처리되도록 하면 됨.
print(competitor_analysis(company))

Received state: {'question': 'sk', 'context': '', 'answer': '', 'messages': [], 'relevance': ''}
==== [RELEVANCE CHECK] ====
결과: yes
### SK 경쟁사 분석 요약 보고서

#### 1. 국내외 경쟁사 리스트 정리
- **반도체 및 전자기기 분야**: 
  - 삼성전자
  - 인텔
  - SK 하이닉스
  - 마이크론
  - 퀄컴
  - 브로드컴
  - 텍사스 인스트루먼트
  - 미디어텍
  - 키옥시아
  - 엔비디아

- **통신 및 서비스 분야**: 
  - KT
  - LG유플러스
  - Verizon (미국)
  - AT&T (미국)

#### 2. 핵심 인사이트 정리
- SK는 반도체와 통신 서비스 두 가지 주요 분야에서 활동하고 있으며, SK 하이닉스는 메모리 반도체 분야에서 두각을 나타내고 있다.
- 삼성전자는 메모리 반도체와 파운드리 사업에서 글로벌 시장 점유율 1위를 차지하고 있으며, 지속적인 기술 혁신을 통해 시장에서의 위치를 강화하고 있다.
- 인텔은 메모리 시장에서 삼성전자와 SK 하이닉스에 밀리며, 최근 비주력 사업을 정리하고 로직 반도체 시장으로의 전환을 도모하고 있다.

#### 3. 경쟁사 대비 전략적 시사점
- **기술 혁신**: SK가 반도체 사업에서 경쟁력을 유지하기 위해서는 삼성전자의 기술 개발 속도를 따라잡아야 하며, AI와 5G와 같은 신시장에 대한 투자 확대가 필요하다.
- **시장 다변화**: SK 하이닉스는 고객 다변화를 통해 위험을 분산시키고, B2B 모델을 더욱 강화해야 한다.
- **인수합병 전략**: 반도체 분야의 경쟁이 치열해짐에 따라, 기술력 있는 기업에 대한 인수합병을 통해 시장 내 입지를 강화할 필요가 있다.

#### 4. 투자측면에서 이점
- **배당 수익률**: SK의 현금배당수익률은 5.50%로, 안정적인 배당 수익을 추구하는 투자자에게 매력적이다.
- **전략적 성장 가능성**: SK 하이